# ECO309 - 2023 : Coursework

- Name(s):

## Part I - Linear Regression

1. __For $N=100$, compute a sample $(x_i, y_i)_{i=[1:N]}$ satisfying $$y_i=0.4+2.5 x_i + \epsilon_i$$ where $x_i$ uniformly distributed between 0 and 1 and $\epsilon_i$ is drawn from a normal distribution with standard deviation 0.5.__




2. __Define the model $f(x;a,b)=a+b x$. Find the parameters $a$ and $b$ which minimize the objective $\xi(a,b)=\sum_i (f(x_i;a,b)-y_i)^2$ by using a numerical optimization algorithm (not the formula for the regression). Plot.__

## Part II - Dividend Pricing

A dividend process is described by $d_t=\exp(\epsilon) (d_{t-1})^\rho$ with $\rho=0.9$ and $\epsilon$ a normal i.i.d. process with mean 0 and standard deviation 0.05.
Given discount rate $\beta=0.95$ our ultimate goal is to evaluate the price of a right to the dividend, which satisfies:

$$p_t=d_t + \beta E_t \left[ p_{t+1} \right]$$

In functional form, we look for a function $p()$ such that

$$\forall d, {\color{blue}{p}}(d)=d+\beta E_{\epsilon} \left[ {\color{red}{p}}\left( \exp(\epsilon) (d)^\rho \right)\right]$$

In this context, the time-iteration algorithm consists in updating the price function on the left as a function of the price function on the right. It raises two challenges:
- integrate the term within the square brackets (we'll use quadrature)
- interpolate the price function so that it can be evaluated at any realisation of $\exp(\epsilon) (d)^\rho$

1. __Integration: discretize random shock $\epsilon_t$ with $K=5$ nodes and weights $(w_i, \epsilon_i)$ by using a gauss-hermite quadrature.
As an example, use this discretization to evaluate $\mathbb{E}\left(\epsilon^2\right)$ and $\mathbb{E}\left[ \exp(\epsilon)\right]$.__

2. __Interpolation: for $N=20$, propose a regular grid $(d_1, ... d_N)$ for $d$ that is large enough to contain a large fraction of of the realizations of $d_t$. Function $p()$ will be represented by an $N$-components vector `p`. Linear interpolation will be used to evaluate `p` outside of the grid points (you can use `Interpolations.jl`). Choose an initial guess for $p$, for instance $p(d)=d^2$ and plot the interpolating function with the interpolated points.__

<mark>Hint</mark>: given a vector of points `x` and matching vector of values `y`, you can use `itp = LinearInterpolation(x, y, extrapolation_bc = Line())` to construct a smooth function, which equals `y` on `x`. You can then interpolate at any point or vector of points `z` with `itp(z)`.


3. __Assuming vector `p0` represents the $p()$ function on the right, compute `p1` vector which represents the $p()$ function on the left. Call `T()` the function which map `p0` to `p1`.__

4. __Find the equilibrium price `p` by iterating on `T()`. Plot.__

## Part III - Neoclassical model and damage function

The goal of this exercise is to compute the optimal transition for a neoclassical growth model, augmented with a climate damage function.
This approach is (loosely) inspired by the work from William Nordhaus on the [DICE model](https://en.wikipedia.org/wiki/DICE_model).


We start with a model characterized by the following equations:

- transition of capital:
$$k_t = (1-\delta) k_{t-1} + i_{t-1}$$
- production: 
$$y_t = e^{a_t}(1-\omega_t)k_t^{\alpha}$$
- budget constraint: $$c_t = y_t - i_t$$

where:
- $k_t$ is aggregate capital
- $a_t$ is the exogenous productivity process
- $\omega_t$ is the damage from climate warming (to be endogenized)

We define the saving rate as $s_t = \frac{i_t}{y_t}$

### Solow model

Initially, we assume that $\omega_t$ is zero.


__1. Create a nameduple `m` to hold the parameters of the model.__

In [5]:
model = (;α=0.3, β=0.96, γ=0.5, δ=0.1)

(α = 0.3, β = 0.96, γ = 0.5, δ = 0.1)

__2. Write `steady_state_residual(model, a::Number, s::Number, v::AbstractVector)::AbstractVector` function to compute the steady-state residuals of the model for any productivity level $a$ and saving rate $s$ where $v$ is a guess for $(k,y,c)$.__

__3. Write a function `steady_state(model, a::Number, s)` to *numerically* compute the steady-state using a nonlinear solver. Compute (and check) the steady-states for $a=0.0$ and $a=0.1$.__

__4. We now assume the productivity process follows the exogenous evolution $a_t=\begin{cases} 0 \; \text{if} \; t=<10 \\ 0.1 \;  \text{if} \; t\geq 10 \end{cases}$. Define vector `a`__




__5. Write a function `solow(model, a::Vector, s::Number)` to compute the transition of all economic variables $(v_1, ... v_T)$ over `T` periods starting from the steady-state, given exogenous productivity process (`length(a)=T`). Choose the return type wisely to make it easy to manipulate. The initial (predetermined) capital should be equal to the steady-state for $a=0$ and you can check that final values should all correspond to the steady-state values for $a=0.1$.__

__6. Plot the results from the solow function and comment.__

### Social planner

With the exact same parameterization we now try to solve the ramsey planner problem. We seek to find the optimal sequence of saving rates $(s_1, ..., s_T)$ so as to maximize the intertemporal value:

$$\max_{s_1, ..., s_T} V(s_1, ..., s_T) = \sum_{t=1}^{T} \beta^t U(c_t)$$ 

where $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ 

(note that in the formula above the periods start at 1 to simplify the coding but this bears no consequence).

The optimality condition from private savings is:

$$\beta E_t \left[ U^{\prime}(c_{t+1}/c_t) \left( (1-\delta + \alpha e^{a_{t+1}} k_{t+1}^{\alpha-1}) \right)\right] = 1

__7. Write a function `steady_state(model, a)` to compute the steady-state for a given productivity level.__

__8. Write a function `ramsey_simul(model, a::Vector, s::AbstractVector)` to simulate the evolution of all variables given a choice of saving rates (with `length(a)=length(s)=T`). Simulate and plot starting from the initial steady-state.__

__9. Write a function `lifetime_reward(model, a::Vector, s::AbstractVector)::Number` which computes the lifetime value of a given choice of saving rates.__

__10. Using your preferred method, compute the gradient of function `u->lifetime_reward(model, a, u,; T=100)`__

__11. Using an algorithm from Optim.jl compute the sequence of saving rates, maximizing utility function. Plot the optimal sequence of saving rates. What happens?__

(hint: despite the fact that the solution is interior, boxed algorithms work better here, for instance with the constraints `0.001<=s<=0.999` ; check documentation to get the api)

__12. Modify the reward function so that the function to maximize is now:__


$$\max_{s_1, ..., s_T} V(s_1, ..., s_T) = \sum_{t=1}^{T} \beta^t U(c_t) + \kappa (k_T - \overline{k})^2$$ 

__where $\kappa$ is a weight (for instance 1000) and $\overline{k}$ the *final* steady-state (i.e. computed for last value of a)__


__Find the optimal sequence of saving rates using numerical optimization.__

__13. Plot the result and comment__

### Damage function

In this section the productivity process is identically zero but we add an energy sector and a damage function.

The production is now: 

$$y_t = e^{a_t}(1-\omega_t) k_{g,t}^{\alpha} e_t^{\alpha_e}$$

where $k^g_t$ is the share of current capital allocated to the production of final goods. Energy $e_t$ is produced using production function

$$e_t = (k_{e,t})^\alpha$$


and capital available $k_t$ (predetermined) must be allocated between the two sectors so that:

$$k_t = k_{g,t} + k_{e,t}$$

Damage function is:

$$\omega_t = \lambda θ_t$$

with $\lambda=0.01$.

Temperature $\theta_t$ follows:

$$θ_t = ρ θ_{t-1} + e_t$$

with $\rho=0.9$ (it should be very autocorrelated but we set it smaller than 1 in order to have a unique steady-state)


__14. Write a new model structure to hold all the old and new parameters__

__15. Assuming market for capital and for energy are perfectly competitive, justify that the fraction of capital allocated to the final sector is $\frac{k_{g,t}}{k_t}=\frac{1}{1+\alpha_e}$.__

__16. Write the new steady-state equations.__

__17. Rewrite a function to compute the steady-state variables for any productivity level, taking climate damage into account.__

__18. Rewrite the `ramsey_simul` and `lifetime_reward` functions taking climate damage into account.__

__19. Solve for the new optimal transition. Plot and comment (compare to the no-damage case)__